# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

## 1. role setting (adding trust relationship)

### 1.1. role check

In [6]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20230712T200237


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [4]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [5]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

In [6]:
!wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/Python/bedrock-runtime.zip ## prev: bedrock-python-sdk.zip
!unzip -o bedrock-runtime.zip -d bedrock-sdk
!rm -rf bedrock-runtime.zip

--2023-09-27 05:48:14--  https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/Python/bedrock-runtime.zip
Resolving d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)... 108.138.82.42, 108.138.82.211, 108.138.82.207, ...
Connecting to d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)|108.138.82.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252012954 (240M) [application/zip]
Saving to: ‘bedrock-runtime.zip.1’

bedrock-runtime.zip 100%[===================>] 240.34M  99.5MB/s    in 2.4s    

2023-09-27 05:48:19 (99.5 MB/s) - ‘bedrock-runtime.zip.1’ saved [252012954/252012954]

Archive:  bedrock-runtime.zip
   creating: bedrock-sdk/reviews/
  inflating: bedrock-sdk/.functional-crt  
  inflating: bedrock-sdk/AWSCLI32.msi  
  inflating: bedrock-sdk/models-starfort-report.json  
  inflating: bedrock-sdk/awscli-bundle.zip  
  inflating: bedrock-sdk/boto3-1.28.55.tar.gz  
  inflating: bedrock-sdk/AWSCLI32PY3.msi  
  inflating: bedrock-sdk/

In [8]:
install_needed = True

In [9]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-*-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain #==0.0.286
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    #!{sys.executable} -m pip install -U urllib
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.3/893.3 kB 9.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.189.0-py2.py3-none-any.whl size=1194905 sha256=7898146c91af9b84ac1c939c47be26f08461fe4014b82caa5acd5cfddc1a6f78
  Stored in directory: /root/.cache/pip/wheels/98/11/11/0ed146622a4b4485d9f3c5454fb42f07895c69b6d1d1516987
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.188.0
    Uninstalling sagemaker-2.188.0:
      Successfully uninstalled sagemaker-2.188.0
Processing ./bedrock-sdk/botocore-*-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/root/amazon-bedrock-workshop-webinar-kr/00_Setup/bedrock-sdk/botocore-*-py3-none-any.whl'

Processing ./bedrock-sdk/boto3-*-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or 

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [1]:
import boto3
import langchain

In [2]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.306
boto3 version check: 1.28.55


In [5]:
is_internal_use = <internal-use> ## if 고객: False, aws직원: True
#is_internal_use = True

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [6]:
bedrock_region = <your region> ## "us-east-1" or "us-west-2"
#bedrock_region = 'us-east-1'

In [7]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock-runtime.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [8]:
bedrock_config

{'region_name': 'us-east-1',
 'endpoint_url': 'https://bedrock-runtime.us-east-1.amazonaws.com'}

In [9]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"]
    ) 

In [10]:
# output_text = bedrock_client.list_foundation_models()
# output_text 

## 5. generation

In [11]:
from termcolor import colored
from langchain.llms.bedrock import Bedrock

### Titan

In [12]:
llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)

In [13]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [14]:
print (colored(llm(prompt), "green"))


SageMaker is a fully managed service from Amazon that provides every developer and data scientist the best practices to build, train, and deploy high-quality machine learning (ML) models quickly. With SageMaker, there is no need to select and provision servers, configure ML infrastructure, or manage storage and security. Just use APIs to create, train, and deploy models fully managed in the cloud.


### Claude v2

In [15]:
llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":[],
        "temperature":0,
        "top_p":0.9
    }
)

In [16]:
prompt = "\n\nHuman: Please let us know SageMaker's advantages in 100 words \n\nAssistant:"

In [17]:
print (colored(llm(prompt), "green"))

 Here is a 100 word summary of some of SageMaker's key advantages:

SageMaker makes it easy to build, train, and deploy machine learning models at scale. It provides pre-built algorithms and frameworks as well as fully managed infrastructure to quickly get models into production. Automated hyperparameter tuning saves time by finding the best model configurations. SageMaker Studio provides a visual IDE for the full ML workflow. And SageMaker Neo compiles models for faster inference. The service is fully managed so you don't have to configure servers or manage infrastructure. These capabilities allow data scientists to focus on building models rather than infrastructure.
